In [271]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import copy
import pandas as pd
from newspaper import Article
import re
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import string
import pandas as pd
import re 
data = pd.read_csv('vocab.csv')
stopwords_ru=list(data['0'])
# символы для удаления
patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

In [272]:
# лемматизация
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
#     print("len=",len(doc.split()))
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    raznost=len(doc.split())-len(tokens)
#     print("/////////////////////////////////////")
#     print(raznost)
#     print(round(0.07*len(doc.split())))
#     print()
    if  raznost >= round(0.1*len(doc.split())):
        return 0
    else:
        return 112

In [273]:
# получить текст новостной статьи и проверяем его на принадлежность теме
def get_news(url, data_,flag):
    news = Article(url, language='ru')
    news.download()
    news.parse()
    data_frame=pd.DataFrame()
    el=str(news.text)
    if flag==1:
        text = copy.deepcopy(el[el.find("\n",el.find("\n")+2)+2:].lower())
    else:
        text=copy.deepcopy(el)
    data_frame=pd.DataFrame()
    data_frame=pd.concat([data_frame,pd.DataFrame([text])])
    flag=data_frame[0].apply(lemmatize)
    if flag[0]==0:
        data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[1],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
#     else:
#         data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[0],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
    return data_
# 0 - текст новости
# 1 - относится ли к ИБ (1-да, 0- нет)
# 2 - является фейком (1-да,0 - нет)
# 3 - ссылка на новость

In [274]:
# выясняем сколько страниц на сайие
def secure_lab_page():
    url="https://www.securitylab.ru/news/page1_1.php"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find('div', class_='page-picker')
    st = str(box.find('input'))
    num_page=''
    for i in range(12,len(st)):
        if st[i]=='"':
            break
        else:
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [275]:
# получаем новости
def secure_lab_news(num_page):
    dataframe=pd.DataFrame()
    for k in range(1,num_page+1):
        url="https://www.securitylab.ru/news/page1_"+str(k)+".php"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
        box = soup.find_all('a', class_='article-card inline-card')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (42,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
                        dataframe=get_news("https://www.securitylab.ru"+urls,dataframe,1)
                        break
                if len(dataframe)>=20:
                    break
            except ValueError:
                e=1
        if len(dataframe)>=20:
                break
        print(len(dataframe)," k=",k)
    dataframe

In [276]:
# выясняем сколько страниц на сайие
def xakep_page():
    url="https://xakep.ru/category/news/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find('span', class_='pages')
    st=str(box)
    num_page=''
    for i in range(34,len(st)):
        if st[i]=='<':
            break
        elif re.search(r'[0-9]', st[i]):
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [277]:
def xakep_news(num_page):
    dataframe=pd.DataFrame()
    for k in range(1,num_page+1):
        url="https://xakep.ru/category/news/page/"+str(k)+"/"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='bdaia-blocks-container'))
        box = soup.find_all('h3',class_='entry-title')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (33,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
                        dataframe=get_news(urls,dataframe,2)
                        break
                if len(dataframe)>=20:
                    break
            except ValueError:
                e=1
        if len(dataframe)>=20:
                break
        print(len(dataframe)," k=",k)
    dataframe

In [278]:
# выясняем сколько страниц на сайие
def anti_malware_page():
    url="https://www.anti-malware.ru/news?page=0"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('li',class_='pager-last'))
    box = soup.find('a')
    st=str(box)
    num_page=''
    for i in range(20,len(st)):
        if st[i]=='"':
            break
        elif re.search(r'[0-9]', st[i]):
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [279]:
def anti_malware_news(num_page):
    dataframe=pd.DataFrame()
    for k in range(0,num_page):
        url="https://www.anti-malware.ru/news?page="+str(k)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='region region-content'))
        box = soup.find_all('h2')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (13,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
                        dataframe=get_news('https://www.anti-malware.ru'+urls,dataframe,2)
                        break
                if len(dataframe)>=20:
                    break
            except ValueError:
                e=1
        if len(dataframe)>=20:
                break
        print(len(dataframe)," k=",k)
    dataframe

In [280]:
# выясняем сколько страниц на сайие
def securitymedia_page():
    url="https://securitymedia.org/news/?PAGEN_2=1"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='modern-page-navigation'))
    box = soup.find_all('a')
    st=str(box[len(box)-2])
    num_page=''
    for i in range(24,len(st)):
        if st[i]=='"':
            break
        else:
            num_page=num_page+st[i]
#     print(num_page)
    num_page=int(num_page)
    return num_page

In [281]:
def securitymedia_news(num_page):
    dataframe=pd.DataFrame()
    for k in range(0,num_page):
        url="https://securitymedia.org/news/?PAGEN_2="+str(k)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='col-md-9'))
        box = soup.find_all('a',class_="text-dark")
        link=[]
        for i in range (0, len(box),2):
            try:
                stroka_url=str(box[i])
                urls=''

                for j in range (27,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
                        dataframe=get_news('https://securitymedia.org'+urls,dataframe,2)
                        break
                if len(dataframe)>=20:
                    break
            except ValueError:
                e=1
        if len(dataframe)>=20:
                break
        print(len(dataframe)," k=",k)
    dataframe

In [283]:
page=securitymedia_page()
securitymedia_news(copy.deepcopy(page))

In [182]:
page=anti_malware_page()
anti_malware_news(copy.deepcopy(page+1))

In [137]:
page=xakep_page()
xakep_news(copy.deepcopy(page))

In [138]:
page=secure_lab_page()
secure_lab_news(copy.deepcopy(page))